# 分而治之
有时，程序变得非常长且难以阅读。我们称之为[意大利面条式代码](https://en.wikipedia.org/wiki/Spaghetti_code)。使代码更易于阅读和维护的一种方法是将其分成更小的函数，并在更复杂的工作流程中使用它们。这种软件设计原则被称为[分而治之](https://www.quora.com/What-is-divide-and-conquer-programming-strategy)。

In [1]:
from skimage.io import imread
from skimage.morphology import white_tophat, disk
from skimage.filters import gaussian, threshold_otsu
from skimage.measure import label, regionprops_table
import pandas as pd
import numpy as np

In [2]:
image = imread("../../data/blobs.tif")
footprint = disk(15)
background_subtracted = white_tophat(image, 
                                     footprint=footprint)
particle_radius = 5
denoised = gaussian(background_subtracted, 
                    sigma=particle_radius)
binary = denoised > threshold_otsu(denoised)
labels = label(binary)
requested_measurements = ["label", "area", "mean_intensity"]
regionprops = regionprops_table(image, 
                                labels, 
                                properties=requested_measurements)
table = pd.DataFrame(regionprops)
mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
mean_total_intensity

17136.90322580645

使这种代码更易读的一种常见且简单的方法是将其分成几个部分，每个部分都以注释开始。

In [3]:
# configuration
file_to_analyze = "../../data/blobs.tif"
background_subtraction_radius = 15
particle_radius = 5
requested_measurements = ["area", "mean_intensity"]

# load data
image = imread(file_to_analyze)

# preprocess image
footprint = disk(background_subtraction_radius)
background_subtracted = white_tophat(image, 
                                     footprint=footprint)
denoised = gaussian(background_subtracted, 
                    sigma=particle_radius)

# segment image
binary = denoised > threshold_otsu(denoised)
labels = label(binary)

# extract features
regionprops = regionprops_table(image, 
                                labels, 
                                properties=requested_measurements)
table = pd.DataFrame(regionprops)

# descriptive statistics
mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
mean_total_intensity

17136.90322580645

更专业的做法是将所有这些代码放入有意义的子程序中，并从一个中心函数调用它们。

In [4]:
# reusable functions
def preprocess_image(image, background_subtraction_radius, particle_radius):
    """Apply background removal and denoising"""
    footprint = disk(background_subtraction_radius)
    background_subtracted = white_tophat(image, footprint=footprint)
    denoised = gaussian(background_subtracted, sigma=particle_radius)
    return denoised

def segment_image(image):
    """Apply thresholding and connected component analysis"""
    binary = image > threshold_otsu(image)
    labels = label(binary)
    return labels

def extract_features(image, labels, requested_measurements):
    """Measure specified properties"""
    regionprops = regionprops_table(image, 
                                    labels, 
                                    properties=requested_measurements)
    table = pd.DataFrame(regionprops)
    return table

在我们将处理步骤分组到函数中之后，我们可以从一个主要函数中调用它们。这个函数以后可以被重用来对文件夹中的所有图像应用相同的处理程序。它也作为索引，提供图像处理工作流程的概览。通过仅阅读这个函数，我们就可以知道所有的处理步骤及其参数。

In [5]:
def analyse_average_total_intensity(filename, 
                                    background_subtraction_radius = 15, 
                                    particle_radius = 5):
    """Load an image, segment objects and measure their mean total intensity."""
    image = imread(filename)
    denoised = preprocess_image(image, 
                                background_subtraction_radius, 
                                particle_radius)
    labels = segment_image(denoised)
    requested_measurements = ["area", "mean_intensity"]
    table = extract_features(image, 
                             labels, 
                             requested_measurements)

    # descriptive statistics
    mean_total_intensity = np.mean(table["area"] * table["mean_intensity"])
    
    return mean_total_intensity

In [6]:
# configuration
file_to_analyze = "../../data/blobs.tif"

这个中心函数现在可以很容易地阅读；它只有6行代码

In [7]:
analyse_average_total_intensity(file_to_analyze)

17136.90322580645

然后，这个函数也可以被重用于其他图像文件。

In [8]:
analyse_average_total_intensity("../../data/BBBC007_batch/20P1_POS0005_D_1UL.tif")

884.2620087336245